## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

In [1]:
import os
import sys
import random
from sklearn.model_selection import train_test_split

## Faz download e carrega o dataset

In [2]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [3]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [4]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [5]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

--Pois vem contar-m'o.

--Preguiçosa!... disse Isabel sorrindo.

--Pery!...

--Tu saberás. Deixa Pery fazer o que tem no pensamento.

A moça ajoelhou e fez uma oração breve; pedia a Deus uma ultima graça; pedia a eternidade e a ventura do seu amor, que tinha passado tão rapido pela terra.

Number of paragraphs: 4892


4892

## Análise do dataset

In [6]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12603

## Criando um vocabulário

In [7]:
# Reduzido o tamanho do vocabulário baseado no exercício anterior.
vocab_size = 7000
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [8]:
print(vocab)

{'a': 1, 'que': 2, 'o': 3, 'de': 4, 'e': 5, 'se': 6, 'um': 7, 'do': 8, 'não': 9, 'uma': 10, 'da': 11, 'os': 12, 'com': 13, 'sua': 14, 'para': 15, 'seu': 16, 'pery': 17, 'as': 18, 'em': 19, 'no': 20, 'por': 21, 'ao': 22, 'como': 23, 'lhe': 24, 'd': 25, 'á': 26, 'tinha': 27, 'era': 28, 'cecilia': 29, 'na': 30, 'é': 31, 'sobre': 32, 'mas': 33, 'elle': 34, 'the': 35, 'dos': 36, 'indio': 37, 'me': 38, 'seus': 39, 'mais': 40, 'antonio': 41, 'quando': 42, 'alvaro': 43, 'disse': 44, 'das': 45, 'vos': 46, 'of': 47, 'ella': 48, 'olhos': 49, 'te': 50, 'senhora': 51, 'menina': 52, 'pela': 53, 'tu': 54, 'depois': 55, 'nos': 56, 'isabel': 57, 'havia': 58, 'gutenberg': 59, 'fidalgo': 60, 'casa': 61, 'estava': 62, 'ainda': 63, 'tempo': 64, 'já': 65, 'mariz': 66, 'project': 67, 'aventureiros': 68, 'momento': 69, 'loredano': 70, 'só': 71, 'mesmo': 72, 'italiano': 73, 'todos': 74, 'pelo': 75, 'vida': 76, 'sem': 77, 'dous': 78, 'to': 79, 'homem': 80, 'eu': 81, 'porque': 82, 'or': 83, 'meio': 84, 'you': 85

In [9]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r'\w+', sentence.lower())]

encode_sentence(cleaned_paragraphs[20], vocab)

[6594,
 139,
 4376,
 19,
 6595,
 6,
 44,
 110,
 269,
 259,
 2662,
 10,
 1064,
 6596,
 2,
 186,
 130,
 280,
 3,
 2257,
 6,
 6597,
 1,
 2665]

## Classe do dataset

In [10]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    # Define your data here
    self.tokens = []
    self.targets = []
    self.removed = 0
    for paragraph in paragraphs:
      encoded = encode_sentence(paragraph, vocab)
      # Do not add examples with unknown tokens
      if 0 not in encoded:
        for i in range(len(encoded) - context):
          self.tokens.append(encoded[i:i+context])
          self.targets.append(encoded[i+context])
      else:
        self.removed+=1

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def __removed__(self):
    return self.removed

In [11]:
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

print("Training samples: " + str(len(train_data)))
print("Validation samples: " + str(len(val_data)))

Training samples: 3913
Validation samples: 979


In [12]:
context_size = 9

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Samples
print("Training samples:")
print(train_dataset[:5])
print()
print("Validation samples:")
print(val_dataset[:5])
print()
print("Removed training samples: " + str(train_dataset.removed))
print("Removed validation samples: " + str(val_dataset.removed))

Training samples:
(tensor([[4082,    2,   25,   41,    4,   66, 4083,  393, 3119],
        [   2,   25,   41,    4,   66, 4083,  393, 3119,   50],
        [  25,   41,    4,   66, 4083,  393, 3119,   50,    3],
        [  41,    4,   66, 4083,  393, 3119,   50,    3,    2],
        [   4,   66, 4083,  393, 3119,   50,    3,    2, 1336]]), tensor([  50,    3,    2, 1336,   21]))

Validation samples:
(tensor([[  31, 1602,  839,    1,  105, 1443,   21,   14,  117],
        [1602,  839,    1,  105, 1443,   21,   14,  117,   55],
        [ 839,    1,  105, 1443,   21,   14,  117,   55,    4],
        [   1,  105, 1443,   21,   14,  117,   55,    4,  953],
        [ 105, 1443,   21,   14,  117,   55,    4,  953, 2330]]), tensor([  55,    4,  953, 2330,  234]))

Removed training samples: 2068
Removed validation samples: 537


In [13]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[  10,   71,  121,    4, 3833,  129,    4, 1333,   22],
        [ 981,   35, 5923,   83, 5924,   47,  184, 5925,   47],
        [3187,   23,    7,  855,    8, 1247, 3113,    3,  100],
        [  10,   45, 1642,    2,    6, 1437,   42,   18,  161],
        [  32,    3,  392, 3168,  693,  203,   68,    3,   16],
        [2750, 1278,  101,    1,  171,    3,  223, 4530,  253],
        [   3,   16, 1769,   33, 4513,    3, 6923,  508,   10],
        [   8,   73,    2, 4242,   12,   68,   26, 1058,  153],
        [1026, 2021,   82,  266, 1304,    4,  641,  276,   54],
        [   1,  882, 1873,    1, 2999,   15,    1, 1885, 2024],
        [2210, 5775,  558,   67,   59,  395, 5776,   85, 5777],
        [  30,   14, 4867, 4847,    7,  313,    2,   15,   34],
        [ 707,    2,   27,  151,  193,  446,    6,  426,   19],
        [  27, 1350,    1,   92,   32,   34,   23,    3,   16],
        [1615,    5,  129,   10,   71,  117,   24, 5589,    1],
        [  10,  258,   15,   10,  617, 

## Model

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.linear2 = nn.Linear(h, vocab_size+1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer with Relu activation
        out = self.linear1(embeds)
        out = F.relu(out)
        # Second layer
        out = self.linear2(out)
        return out

In [15]:
embedding_dim = 64
hidden_dim = 128
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)

In [16]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

torch.Size([32, 9])
torch.Size([32])


In [17]:
output = model(input)

In [18]:
output.argmax(dim=1)

tensor([3938,  712,  499, 2949, 3165,  846, 4949, 4256, 1997, 4468,  915, 5127,
        2361, 1764, 2600,  817,  846, 1210, 5718, 1846,  179, 6299, 5818, 5372,
        4639, 1997, 3598, 4737, 3173, 1833, 5205,  137])

In [19]:
target

tensor([   2, 4689, 1407,  200, 4397,   15, 3012,   83,  193,   20,    7,    5,
         730,   43,    1, 1936,   42, 5064,   54,  294,    4, 3337,  236,  109,
          31,  584,   17,  538,  146,  524, 5979,  962])

## Training

In [20]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [21]:
epochs = 10

# Learning rate
lr = 0.01

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr)

model.to(device)

BengioModel(
  (embeddings): Embedding(7001, 64)
  (linear1): Linear(in_features=576, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=7001, bias=True)
)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exemplo de uso:
total_params = count_parameters(model)
print(f'O modelo tem um total de {total_params:,} parâmetros.')

O modelo tem um total de 1,425,049 parâmetros.


In [23]:
# Initial Perplexity and Loss
# Before training
model.eval()

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

loss /= len(train_loader)
perp = torch.exp(torch.tensor(loss))

print(f'Initial Loss: {loss:.4f}')
print(f'Initial Perplexity: {perp:.4f}')

Initial Loss: 8.8761
Initial Perplexity: 7158.4727


In [24]:
# Training Loop

for epoch in range(epochs):
  model.train()

  # Metrics
  epoch_loss = 0
  epoch_correct = 0
  epoch_samples = 0

  for inputs, targets in train_loader:
        inputs = inputs.to(device)  # Move input data to the device
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss
        epoch_loss += loss.item()

        # Predicted
        _, predicted = torch.max(outputs, 1)
        epoch_correct += (predicted == targets).sum().item()
        epoch_samples += targets.size(0)

  # Calculate average loss and accuracy for epoch
  avg_loss = epoch_loss / len(train_loader)
  acc = epoch_correct / epoch_samples

  # Perplexity
  perp = torch.exp(torch.tensor(avg_loss))

  # Print epoch statistics
  print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


Epoch [1/10], Loss: 7.5764, Accuracy: 0.05%, Perplexity: 1951.6860
Epoch [2/10], Loss: 6.4637, Accuracy: 0.08%, Perplexity: 641.4094
Epoch [3/10], Loss: 6.2345, Accuracy: 0.09%, Perplexity: 510.0681
Epoch [4/10], Loss: 6.1315, Accuracy: 0.10%, Perplexity: 460.1183
Epoch [5/10], Loss: 6.0817, Accuracy: 0.11%, Perplexity: 437.7627
Epoch [6/10], Loss: 6.0900, Accuracy: 0.11%, Perplexity: 441.4312
Epoch [7/10], Loss: 6.0900, Accuracy: 0.11%, Perplexity: 441.4013
Epoch [8/10], Loss: 6.0822, Accuracy: 0.11%, Perplexity: 438.0112
Epoch [9/10], Loss: 6.0971, Accuracy: 0.11%, Perplexity: 444.5545
Epoch [10/10], Loss: 6.0442, Accuracy: 0.12%, Perplexity: 421.6741


## Avaliação

In [25]:
import torch.nn.functional as F

model.eval()

loss_sum = 0
total_sum = 0
correct_sum = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, targets)
        loss_sum += loss

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        total_sum += targets.size(0)
        correct_sum += (predicted == targets).sum().item()

# Calculate accuracy
acc = 100 * correct_sum/total_sum

# Calculate average perplexity
average_loss = loss_sum / len(val_loader)
average_perplexity = torch.exp(torch.tensor(average_loss))

print(f'Test Accuracy: {acc}%')
print(f'Average Perplexity: {average_perplexity}')

Test Accuracy: 7.828337287039034%
Average Perplexity: 3931.050048828125


C:\Users\fabiogr\AppData\Local\Temp\ipykernel_2964\282167542.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  average_perplexity = torch.exp(torch.tensor(average_loss))


## Exemplo de uso

In [26]:
# Código adaptado da implementação do Cesar Bastos
from colorama import Fore, Style

text = cleaned_paragraphs
model.to(device)
def generate_text(model, vocab, text, max_length, context_size):
    words = []
    # Ensure there are enough words for at least one sequence
    while len(words) < context_size:
        random_number = random.randint(1, 4891)
        words = encode_sentence(text[random_number], vocab)
        if not words:
            words = []
            continue  # Skip if the sentence cannot be encoded
        words = words[:context_size]
        #print(words)
        if any(token == 0 for token in words):
            words = []
            continue  # Skip if any token is zero (assuming 0 is a special token)
        context = words

    print(f"Frase: {cleaned_paragraphs[random_number]}")
    print(words)

    for _ in range(max_length):
        words_tensor = torch.tensor(context[-context_size:], dtype=torch.long).unsqueeze(0).to(device)
        logits = model(words_tensor)
        probs = F.softmax(logits, dim=1)
        next_token = torch.multinomial(probs, num_samples=1)
        context.append(next_token.item())
        print(context)
    frase = []
    for i in context: ##Agradecimentos a Ramon Abilio
        word = next((word for word, code in vocab.items() if code == i), "<UNKNOWN>")
        frase.append(word)

    print(f"{Fore.BLUE}{frase[:context_size]}{Style.RESET_ALL} {Fore.RED}{frase[-max_length:]}{Style.RESET_ALL} ")

context_size = 9
max_length= 10
generate_text(model, vocab, text, max_length, context_size)

Frase: --Dentro de uma hora, continuou o cavalheiro apontando para o corpo de Loredano, este homem será justiçado á frente da banda; para elle não ha julgamento; eu o condemno como pai, como chefe, como um homem que mata o cão ingrato que o morde. É ignobil de mais para que o toque com as minhas armas; entrego-o ao baraço e ao cutelo.
[567, 4, 10, 192, 353, 3, 113, 1291, 15]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875, 2]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875, 2, 14]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875, 2, 14, 451]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875, 2, 14, 451, 3]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875, 2, 14, 451, 3, 5]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875, 2, 14, 451, 3, 5, 23]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260, 2875, 2, 14, 451, 3, 5, 23, 2718]
[567, 4, 10, 192, 353, 3, 113, 1291, 15, 3260,